# CubeDynamics Quickstart: Pipe-based Climate Cube Operations

Install CubeDynamics from GitHub (run in your terminal or a notebook cell before importing):

```bash
pip install "git+https://github.com/CU-ESIIL/climate_cube_math.git@main"
```

If you prefer running inside the notebook, uncomment and execute the following cell: `!pip install "git+https://github.com/CU-ESIIL/climate_cube_math.git@main"`.

In [ ]:
# Optional notebook install
# !pip install "git+https://github.com/CU-ESIIL/climate_cube_math.git@main"

In [ ]:
import numpy as np
import xarray as xr
import cubedynamics as cd

## Minimal pipe chain

Create a tiny 1D cube, compute anomalies, and then variance along time.

In [ ]:
time = np.arange(6)
values = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])

cube = xr.DataArray(
    values,
    dims=["time"],
    coords={"time": time},
    name="example_variable",
)

result = (
    cd.pipe(cube)
    | cd.anomaly(dim="time")
    | cd.variance(dim="time")
).unwrap()

result

## Filter months and write to NetCDF

This example filters the summer months from a synthetic monthly series and writes them to a temporary NetCDF file.

In [ ]:
time_months = np.arange("2000-01", "2001-01", dtype="datetime64[M]")
values_months = np.arange(time_months.size)
monthly_cube = xr.DataArray(
    values_months,
    dims=["time"],
    coords={"time": time_months},
    name="monthly_example",
)

from pathlib import Path
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    out_path = Path(tmpdir) / "summer.nc"
    summer = (
        cd.pipe(monthly_cube)
        | cd.month_filter([6, 7, 8])
        | cd.to_netcdf(out_path)
    ).unwrap()

    print("Saved summer subset to:", out_path)
    summer